<a href="https://colab.research.google.com/github/tilly963963/pytorch_nlp/blob/main/7_3_Model1_Unconditioned_Surname_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surname Generation

## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/Shareddrives/python教學/PyTorchNLPBook-master/chapters/chapter_7/7_3_surname_generation")

In [3]:
import os
from argparse import Namespace
from collections import Counter
import json
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
import time

## Data Vectorization classes

### Vocabulary

In [4]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """
        print("Vocabulary __init__")
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        print("self._token_to_idx=",self._token_to_idx)
        print("self._idx_to_token=",self._idx_to_token) 
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        # print(token,":",index)
        return index
            
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [5]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):
        print("SequenceVocabulary __init__")
        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        print("contents=",contents)
        # contents= {'token_to_idx': {'<MASK>': 0, '<UNK>': 1, '<BEGIN>': 2, 
        # '<END>': 3, 'T': 4, 'o': 5, 't': 6, 'a': 7, 'h': 8, 'A': 9, 'b': 10, 
        # 'u': 11, 'd': 12, 'F': 13, 'k': 14, 'r': 15, 'y': 16, 'S': 17, 'e': 18, 
        # 'g': 19, 'C': 20, 'm': 21, 'H': 22, 'i': 23, 'K': 24, 'n': 25, 'W': 26, 
        # 's': 27, 'f': 28, 'G': 29, 'M': 30, 'l': 31, 'B': 32, 'z': 33, 'N': 34,
        # 'I': 35, 'w': 36, 'D': 37, 'Q': 38, 'j': 39, 'E': 40, 'R': 41, 'Z': 42,
        # 'c': 43, 'Y': 44, 'J': 45, 'L': 46, 'O': 47, '-': 48, 'P': 49, 'X': 50,
        # 'p': 51, ':': 52, 'v': 53, 'U': 54, '1': 55, 'V': 56, 'x': 57, '/': 58,
        # 'q': 59, 'é': 60, 'É': 61, "'": 62, 'ç': 63, 'ê': 64, 'ß': 65,
        # 'ö': 66, 'ä': 67, 'ü': 68, 'ú': 69, 'à': 70, 'ò': 71, 'è': 72, 
        # 'ó': 73, 'ù': 74, 'ì': 75, 'Ś': 76, 'ą': 77, 'ń': 78, 'á': 79, 
        # 'ż': 80, 'Ż': 81, 'ł': 82, 'õ': 83, 'ã': 84, 'í': 85, 'ñ': 86, 'Á': 87},
        # unk_token': '<UNK>', 'mask_token': '<MASK>', 'begin_seq_token': '<BEGIN>', 'end_seq_token': '<END>'}
        
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

### Vectorizer

In [6]:
class SurnameVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, char_vocab, nationality_vocab):
        """
        Args:
            char_vocab (Vocabulary): maps words to integers
            nationality_vocab (Vocabulary): maps nationalities to integers
        """
        print("SurnameVectorizer __init__")
        self.char_vocab = char_vocab
        self.nationality_vocab = nationality_vocab

    def vectorize(self, surname, vector_length=-1):
        """Vectorize a surname into a vector of observations and targets
        
        The outputs are the vectorized surname split into two vectors:
            surname[:-1] and surname[1:]
        At each timestep, the first vector is the observation and the second vector is the target. 
        
        Args:
            surname (str): the surname to be vectorized
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            a tuple: (from_vector, to_vector)
            from_vector (numpy.ndarray): the observation vector 
            to_vector (numpy.ndarray): the target prediction vector
        """
        print("surname=",surname)
        indices = [self.char_vocab.begin_seq_index] 
        indices.extend(self.char_vocab.lookup_token(token) for token in surname)
        indices.append(self.char_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices) - 1

        from_vector = np.empty(vector_length, dtype=np.int64) 
        # print("indices=",indices,"vector_length=",vector_length)        
        from_indices = indices[:-1]
        from_vector[:len(from_indices)] = from_indices
        from_vector[len(from_indices):] = self.char_vocab.mask_index

        to_vector = np.empty(vector_length, dtype=np.int64)
        to_indices = indices[1:]
        to_vector[:len(to_indices)] = to_indices
        to_vector[len(to_indices):] = self.char_vocab.mask_index
        # print("from_vector=",from_vector,"to_vector=",to_vector)
# from_vector= [2 4 5 6 7 8 0 0 0 0 0 0 0 0 0 0 0 0 0] to_vector= [4 5 6 7 8 3 0 0 0 0 0 0 0 0 0 0 0 0 0]

# from_vector= [ 2  9 10 10  5 11 12  0  0  0  0  0  0  0  0  0  0  0  0] to_vector= [ 9 10 10  5 11 12  3  0  0  0  0  0  0  0  0  0  0  0  0]

        return from_vector, to_vector

    @classmethod
    def from_dataframe(cls, surname_df):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            surname_df (pandas.DataFrame): the surname dataset
        Returns:
            an instance of the SurnameVectorizer
        """
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()
        # SequenceVocabulary __init__
        #     繼承Vocabulary __init__
        # self._token_to_idx= {}
        # self._idx_to_token= {}
        # <MASK> : 0
        # <UNK> : 1
        # <BEGIN> : 2
        # <END> : 3
        # Vocabulary __init__
        # self._token_to_idx= {}
        # self._idx_to_token= {}
        for index, row in surname_df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
                # time.sleep(10)
                # T : 4
                # o : 5
                # t : 6
                # a : 7
                # h : 8
            nationality_vocab.add_token(row.nationality)
            # Arabic : 0            
            
            if index == 0:
              print("row=",row)
              print("char_vocab=",char_vocab)
              print("nationality_vocab=",nationality_vocab)
              # row= nationality          Arabic
              # nationality_index        15
              # split                 train
              # surname               Totah
              # Name: 0, dtype: object
              # char_vocab= <Vocabulary(size=9)>
              # nationality_vocab= <Vocabulary(size=1)>
        return cls(char_vocab, nationality_vocab)#SurnameVectorizer __init__



    @classmethod
    def from_serializable(cls, contents):
        """Instantiate the vectorizer from saved contents
        
        Args:
            contents (dict): a dict holding two vocabularies for this vectorizer
                This dictionary is created using `vectorizer.to_serializable()`
        Returns:
            an instance of SurnameVectorizer
        """
        char_vocab = SequenceVocabulary.from_serializable(contents['char_vocab'])
        nat_vocab =  Vocabulary.from_serializable(contents['nationality_vocab'])

        return cls(char_vocab=char_vocab, nationality_vocab=nat_vocab)

    def to_serializable(self):
        """ Returns the serializable contents """
        print('char_vocab', self.char_vocab.to_serializable())
        print('nationality_vocab', self.nationality_vocab.to_serializable())
        return {'char_vocab': self.char_vocab.to_serializable(), 
                'nationality_vocab': self.nationality_vocab.to_serializable()}

### Dataset

In [7]:
class SurnameDataset(Dataset):
    def __init__(self, surname_df, vectorizer):
        """
        Args:
            surname_df (pandas.DataFrame): the dataset
            vectorizer (SurnameVectorizer): vectorizer instatiated from dataset
        """
        self.surname_df = surname_df 
        self._vectorizer = vectorizer

        self._max_seq_length = max(map(len, self.surname_df.surname)) + 2
        print("self._max_seq_length=",self._max_seq_length)
        # self._max_seq_length= 19

        self.train_df = self.surname_df[self.surname_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.surname_df[self.surname_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.surname_df[self.surname_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size), 
                             'val': (self.val_df, self.validation_size), 
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, surname_csv):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            surname_csv (str): location of the dataset
        Returns:
            an instance of SurnameDataset
        """
        
        surname_df = pd.read_csv(surname_csv)
        print("surname_df=")
        print(surname_df)
#         surname_df=
#       nationality  nationality_index  split   surname
# 0          Arabic                 15  train     Totah
# 1          Arabic                 15  train    Abboud
# 2          Arabic                 15  train  Fakhoury
# 3          Arabic                 15  train     Srour
# 4          Arabic                 15  train    Sayegh
# ...           ...                ...    ...       ...
# 10975  Vietnamese                 11   test      Dinh
# 10976  Vietnamese                 11   test     Phung
# 10977  Vietnamese                 11   test     Quang
# 10978  Vietnamese                 11   test        Vu
# 10979  Vietnamese                 11   test        Ha

# [10980 rows x 4 columns]
        return cls(surname_df, SurnameVectorizer.from_dataframe(surname_df))
        
    @classmethod
    def load_dataset_and_load_vectorizer(cls, surname_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            surname_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of SurnameDataset
        """
        surname_df = pd.read_csv(surname_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(surname_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of SurnameVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return SurnameVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point: (x_data, y_target, class_index)
        """
        print("__getitem__")
        row = self._target_df.iloc[index]
        
        from_vector, to_vector = \
            self._vectorizer.vectorize(row.surname, self._max_seq_length)
        # print("from_vector=",from_vector,"to_vector=",to_vector)
        nationality_index = \
            self._vectorizer.nationality_vocab.lookup_token(row.nationality)
        # print("nationality_index=",nationality_index)
# from_vector= [2 4 5 6 7 8 0 0 0 0 0 0 0 0 0 0 0 0 0] to_vector= [4 5 6 7 8 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
# from_vector= [2 4 5 6 7 8 0 0 0 0 0 0 0 0 0 0 0 0 0] to_vector= [4 5 6 7 8 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
# nationality_index= 0
# from_vector= [ 2  9 10 10  5 11 12  0  0  0  0  0  0  0  0  0  0  0  0] to_vector= [ 9 10 10  5 11 12  3  0  0  0  0  0  0  0  0  0  0  0  0]
# from_vector= [ 2  9 10 10  5 11 12  0  0  0  0  0  0  0  0  0  0  0  0] to_vector= [ 9 10 10  5 11 12  3  0  0  0  0  0  0  0  0  0  0  0  0]
# nationality_index= 0
        return {'x_data': from_vector, 
                'y_target': to_vector, 
                'class_index': nationality_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size
    
def generate_batches(dataset, batch_size, shuffle=False,
                     drop_last=True, device="cpu"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)
    '''
    當迭代dataloader時會自動呼叫 
    class DataLoader(object)  
      def __iter__(self): 
        return DataLoaderIter(self)
        
    class DataLoaderIter(object): 
      def __next__(self):
        batch = self.collate_fn([self.dataset[i] for i in indices])
        self.dataset[i] => __getitem__() 每一次回傳批次個  用self.collate_fn 打包成批次個
    '''
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
            # time.sleep(10)
        yield out_data_dict

## The Model: SurnameGenerationModel

In [8]:
class SurnameGenerationModel(nn.Module):
    def __init__(self, char_embedding_size, char_vocab_size, rnn_hidden_size, 
                 batch_first=True, padding_idx=0, dropout_p=0.5):
        """
        Args:
            char_embedding_size (int): The size of the character embeddings
            char_vocab_size (int): The number of characters to embed
            rnn_hidden_size (int): The size of the RNN's hidden state
            batch_first (bool): Informs whether the input tensors will 
                have batch or the sequence on the 0th dimension
            padding_idx (int): The index for the tensor padding; 
                see torch.nn.Embedding
            dropout_p (float): the probability of zeroing activations using
                the dropout method.  higher means more likely to zero.
        """
        # model = SurnameGenerationModel(char_embedding_size=args.char_embedding_size,#32
        #                        char_vocab_size=len(vectorizer.char_vocab),#88
        #                        rnn_hidden_size=args.rnn_hidden_size,#32
        #                        padding_idx=vectorizer.char_vocab.mask_index)#0
        
        super(SurnameGenerationModel, self).__init__()
        
        self.char_emb = nn.Embedding(num_embeddings=char_vocab_size,
                                     embedding_dim=char_embedding_size,
                                     padding_idx=padding_idx)

        self.rnn = nn.GRU(input_size=char_embedding_size, 
                          hidden_size=rnn_hidden_size,
                          batch_first=batch_first)
        
        self.fc = nn.Linear(in_features=rnn_hidden_size, 
                            out_features=char_vocab_size)
        
        self._dropout_p = dropout_p

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the model
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, input_dim)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, char_vocab_size)
        """
        print("x_in.shape=",x_in.shape)# x_in.shape= torch.Size([2, 19])
        x_embedded = self.char_emb(x_in)
        print("x_embedded.shape=",x_embedded.shape)# x_embedded.shape= torch.Size([2, 19, 32])
        y_out, _ = self.rnn(x_embedded)
        print("y_out_1.shape",y_out.shape)# y_out_1.shape torch.Size([2, 19, 32])
        print("_.shape",_.shape)# _.shape torch.Size([1, 2, 32])

        batch_size, seq_size, feat_size = y_out.shape
        y_out = y_out.contiguous().view(batch_size * seq_size, feat_size)
        print("y_out_2.shape",y_out.shape)# y_out_2.shape torch.Size([38, 32])
        y_out = self.fc(F.dropout(y_out, p=self._dropout_p))
        print("y_out_3.shape",y_out.shape)# y_out_3.shape torch.Size([38, 88])                 
        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)
            print("y_out_4.shape",y_out.shape)
        new_feat_size = y_out.shape[-1]
        y_out = y_out.view(batch_size, seq_size, new_feat_size)
        print("y_out_5.shape",y_out.shape) # y_out_5.shape torch.Size([2, 19, 88])
        
        
        
        
        
        
           
        return y_out

In [9]:
def sample_from_model(model, vectorizer, num_samples=1, sample_size=20, 
                      temperature=1.0):
    """Sample a sequence of indices from the model
    
    Args:
        model (SurnameGenerationModel): the trained model
        vectorizer (SurnameVectorizer): the corresponding vectorizer
        num_samples (int): the number of samples
        sample_size (int): the max length of the samples
        temperature (float): accentuates or flattens 
            the distribution. 
            0.0 < temperature < 1.0 will make it peakier. 
            temperature > 1.0 will make it more uniform
    Returns:
        indices (torch.Tensor): the matrix of indices; 
        shape = (num_samples, sample_size)
    """
    
    begin_seq_index = [vectorizer.char_vocab.begin_seq_index 
                       for _ in range(num_samples)]
    begin_seq_index = torch.tensor(begin_seq_index, 
                                   dtype=torch.int64).unsqueeze(dim=1)
    indices = [begin_seq_index]
    print("indices=",indices)
    h_t = None
    # indices= [tensor([[2],[2]])]
    # --time_step 0
    # x_t= torch.Size([2, 1])
    # x_emb_t= torch.Size([2, 1, 32])
    # rnn_out_t= torch.Size([2, 1, 32])
    # x_emb_t= torch.Size([2, 1, 32]) h_t= torch.Size([1, 2, 32])
    # prediction_vector= torch.Size([2, 88])
    # probability_vector= torch.Size([2, 88])

    for time_step in range(sample_size):
        print("--time_step",time_step)
        x_t = indices[time_step]
        print(" x_t=",x_t.size(),"x_t=",x_t)
        #torch.Size([2, 1])
        x_emb_t = model.char_emb(x_t)
        print(" x_emb_t=",x_emb_t.size())#torch.Size([2, 1, 32])
        # print(" x_emb_t=",x_emb_t)
        # x_emb_t= tensor([[[-1.0201,  0.5903, -0.2879, -0.0490, -0.5148,  0.3801, -0.1592,
        #   -1.5006,  2.3597, -0.5313,  0.2675,  0.7611, -1.5452,  1.2178,
        #    0.4369,  0.5798,  2.8189, -1.4704,  1.3872,  0.7294, -1.8940,
        #   -1.3537,  0.8752, -0.1787,  0.6776, -0.4856,  0.7808, -0.6946,
        #   -0.4361, -0.2666, -1.1842, -0.4199]],

        # [[-1.0201,  0.5903, -0.2879, -0.0490, -0.5148,  0.3801, -0.1592,
        #   -1.5006,  2.3597, -0.5313,  0.2675,  0.7611, -1.5452,  1.2178,
        #    0.4369,  0.5798,  2.8189, -1.4704,  1.3872,  0.7294, -1.8940,
        #   -1.3537,  0.8752, -0.1787,  0.6776, -0.4856,  0.7808, -0.6946,
        #   -0.4361, -0.2666, -1.1842, -0.4199]]], grad_fn=<EmbeddingBackward>)
        rnn_out_t, h_t = model.rnn(x_emb_t, h_t)
        print(" rnn_out_t=",rnn_out_t.size())#torch.Size([2, 1, 32])
        print("h_t=",h_t.shape)#torch.Size([1, 2, 32])

        prediction_vector = model.fc(rnn_out_t.squeeze(dim=1))
        print(" prediction_vector=",prediction_vector.size())# prediction_vector= torch.Size([2, 88])
        probability_vector = F.softmax(prediction_vector / temperature, dim=1)
        print(" probability_vector=",probability_vector.size())
        
        # probability_vector= torch.Size([2, 88])
        print("probability_vector=",probability_vector)
      #   probability_vector= tensor([[1.0694e-04, 1.0777e-04, 1.2412e-04, 6.3120e-05, 6.6927e-02, 6.8920e-04,
      #    9.2951e-04, 1.1999e-03, 2.3119e-03, 8.4990e-02, 1.5605e-03, 7.1253e-04,
      #    9.8443e-04, 1.7937e-02, 8.8583e-04, 6.7743e-04, 3.1892e-04, 4.9511e-02,
      #    6.1856e-04, 8.2312e-04, 2.9544e-02, 9.2597e-04, 3.7178e-02, 5.9546e-04,
      #    2.5327e-02, 3.3935e-04, 8.6664e-03, 5.9821e-04, 5.5917e-04, 5.6714e-02,
      #    9.3515e-02, 9.9020e-04, 6.6330e-02, 4.2033e-04, 2.3351e-02, 1.2564e-02,
      #    1.1285e-03, 4.8379e-02, 2.0921e-03, 6.2832e-04, 2.5190e-02, 4.3055e-02,
      #    3.2194e-02, 6.5972e-04, 1.9251e-02, 3.7250e-02, 4.0627e-02, 1.9201e-02,
      #    3.6457e-04, 4.6290e-02, 6.4778e-04, 5.3203e-04, 1.5884e-04, 1.0717e-04,
      #    9.3064e-03, 2.5139e-04, 7.7255e-02, 5.7183e-04, 1.3947e-04, 4.7004e-04,
      #    3.6909e-04, 1.9006e-04, 2.8807e-04, 1.2280e-04, 1.4040e-04, 3.4943e-04,
      #    2.6814e-04, 2.5259e-04, 3.1820e-04, 2.0092e-04, 1.2231e-04, 2.1767e-04,
      #    2.0469e-04, 1.4192e-04, 1.1275e-04, 1.4141e-04, 2.3332e-04, 1.7815e-04,
      #    1.2254e-04, 1.1657e-04, 1.6049e-04, 9.7397e-05, 1.1651e-04, 1.0813e-04,
      #    1.2823e-04, 1.4578e-04, 1.2104e-04, 1.8295e-04],
      #   [1.0694e-04, 1.0777e-04, 1.2412e-04, 6.3120e-05, 6.6927e-02, 6.8920e-04,
      #    9.2951e-04, 1.1999e-03, 2.3119e-03, 8.4990e-02, 1.5605e-03, 7.1253e-04,
      #    9.8443e-04, 1.7937e-02, 8.8583e-04, 6.7743e-04, 3.1892e-04, 4.9511e-02,
      #    6.1856e-04, 8.2312e-04, 2.9544e-02, 9.2597e-04, 3.7178e-02, 5.9546e-04,
      #    2.5327e-02, 3.3935e-04, 8.6664e-03, 5.9821e-04, 5.5917e-04, 5.6714e-02,
      #    9.3515e-02, 9.9020e-04, 6.6330e-02, 4.2033e-04, 2.3351e-02, 1.2564e-02,
      #    1.1285e-03, 4.8379e-02, 2.0921e-03, 6.2832e-04, 2.5190e-02, 4.3055e-02,
      #    3.2194e-02, 6.5972e-04, 1.9251e-02, 3.7250e-02, 4.0627e-02, 1.9201e-02,
      #    3.6457e-04, 4.6290e-02, 6.4778e-04, 5.3203e-04, 1.5884e-04, 1.0717e-04,
      #    9.3064e-03, 2.5139e-04, 7.7255e-02, 5.7183e-04, 1.3947e-04, 4.7004e-04,
      #    3.6909e-04, 1.9006e-04, 2.8807e-04, 1.2280e-04, 1.4040e-04, 3.4943e-04,
      #    2.6814e-04, 2.5259e-04, 3.1820e-04, 2.0092e-04, 1.2231e-04, 2.1767e-04,
      #    2.0469e-04, 1.4192e-04, 1.1275e-04, 1.4141e-04, 2.3332e-04, 1.7815e-04,
      #    1.2254e-04, 1.1657e-04, 1.6049e-04, 9.7397e-05, 1.1651e-04, 1.0813e-04,
      #    1.2823e-04, 1.4578e-04, 1.2104e-04, 1.8295e-04]],
      #  grad_fn=<SoftmaxBackward>)
        indices.append(torch.multinomial(probability_vector, num_samples=1))#作用是对input的每一行做n_samples次取值，输出的张量是每一次取值时input张量对应行的下标
        print(" np.array(indices).shape=",indices)    
        # np.array(indices).shape= [tensor([[2],
            # [2]]), tensor([[46],
            # [24]])]
    aa=torch.stack(indices)
    print("torch.stack(indices)=",aa.shape)# torch.Size([21, 2, 1])
    indices = torch.stack(indices).squeeze().permute(1, 0)
    print("indices.size()",indices.size())#indices.size() torch.Size([2, 21])
    # sampled_indices= tensor([[ 2, 46,  7, 27, 10,  7,  5,  3, 25,  3,  3,  3,  3,  3,  6, 33,  3,  3,
    #       3,  7,  3],
    #     [ 2, 24, 18, 25, 11,  3,  6,  5, 25,  3, 18, 15,  3,  3,  3,  3,  3,  3,
    #       7,  3,  3]])
    return indices

def decode_samples(sampled_indices, vectorizer):
    """Transform indices into the string form of a surname
    
    Args:
        sampled_indices (torch.Tensor): the inidces from `sample_from_model`
        vectorizer (SurnameVectorizer): the corresponding vectorizer
    """
    decoded_surnames = []
    vocab = vectorizer.char_vocab
    print("sampled_indices=",sampled_indices)

    # sampled_indices= tensor([[ 2, 46,  7, 27, 10,  7,  5,  3, 25,  3,  3,  3,  3,  3,  6, 33,  3,  3,
    #       3,  7,  3],
    #     [ 2, 24, 18, 25, 11,  3,  6,  5, 25,  3, 18, 15,  3,  3,  3,  3,  3,  3,
    #       7,  3,  3]])
    print("vocab.begin_seq_index=",vocab.begin_seq_index,"vocab.end_seq_index=",vocab.end_seq_index)
    for sample_index in range(sampled_indices.shape[0]):
        surname = ""
        for time_step in range(sampled_indices.shape[1]):
            sample_item = sampled_indices[sample_index, time_step].item()
            print("sample_item=",sample_item)
            if sample_item == vocab.begin_seq_index:
                continue
            elif sample_item == vocab.end_seq_index:
                break
            else:
                surname += vocab.lookup_index(sample_item)
                print(surname)
        decoded_surnames.append(surname)
    print("decoded_surnames=",decoded_surnames)
    return decoded_surnames
# sample_item= 2
# sample_item= 46
# L
# sample_item= 7
# La
# sample_item= 27
# Las
# sample_item= 10
# Lasb
# sample_item= 7
# Lasba
# sample_item= 5
# Lasbao
# sample_item= 3
# sample_item= 2
# sample_item= 24
# K
# sample_item= 18
# Ke
# sample_item= 25
# Ken
# sample_item= 11
# Kenu
# sample_item= 3
# decoded_surnames= ['Lasbao', 'Kenu']

# decoded_surnames= ['Lasbao', 'Kenu']
            

## Training Routine

### Helper functions

In [10]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.
    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better
    
    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
         
        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def normalize_sizes(y_pred, y_true):
    """Normalize tensor sizes
    
    Args:
        y_pred (torch.Tensor): the output of the model
            If a 3-dimensional tensor, reshapes to a matrix
        y_true (torch.Tensor): the target predictions
            If a matrix, reshapes to be a vector
    """
    # print("y_pred.size()=",y_pred.size(),"y_true.size()=",y_true.size())
    # y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
    if len(y_pred.size()) == 3:
        y_pred = y_pred.contiguous().view(-1, y_pred.size(2))
    if len(y_true.size()) == 2:
        y_true = y_true.contiguous().view(-1)
    # print("y_pred.size() out =",y_pred.size(),"y_true.size()=",y_true.size())
    # y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
    return y_pred, y_true

def compute_accuracy(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.max(dim=1)
    
    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()
    
    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100

def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

### General utilities

In [11]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

### Settings and some prep work

In [12]:
args = Namespace(
    # Data and Path information
    surname_csv="data/surnames/surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch7/model1_unconditioned_surname_generation",
    # Model hyper parameters
    char_embedding_size=32,
    rnn_hidden_size=32,
    # Training hyper parameters
    seed=1337,
    learning_rate=0.001,
    batch_size=2,
    num_epochs=100,
    early_stopping_criteria=5,
    # Runtime options
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
    
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
    
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

Expanded filepaths: 
	model_storage/ch7/model1_unconditioned_surname_generation/vectorizer.json
	model_storage/ch7/model1_unconditioned_surname_generation/model.pth
Using CUDA: True


### Initializations

In [13]:
if args.reload_from_files:
    # training from a checkpoint
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv,
                                                              args.vectorizer_file)
else:
    # create dataset and vectorizer
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()

model = SurnameGenerationModel(char_embedding_size=args.char_embedding_size,#32
                               char_vocab_size=len(vectorizer.char_vocab),#88
                               rnn_hidden_size=args.rnn_hidden_size,#32
                               padding_idx=vectorizer.char_vocab.mask_index)#0

surname_df=
      nationality  nationality_index  split   surname
0          Arabic                 15  train     Totah
1          Arabic                 15  train    Abboud
2          Arabic                 15  train  Fakhoury
3          Arabic                 15  train     Srour
4          Arabic                 15  train    Sayegh
...           ...                ...    ...       ...
10975  Vietnamese                 11   test      Dinh
10976  Vietnamese                 11   test     Phung
10977  Vietnamese                 11   test     Quang
10978  Vietnamese                 11   test        Vu
10979  Vietnamese                 11   test        Ha

[10980 rows x 4 columns]
SequenceVocabulary __init__
Vocabulary __init__
self._token_to_idx= {}
self._idx_to_token= {}
Vocabulary __init__
self._token_to_idx= {}
self._idx_to_token= {}
row= nationality          Arabic
nationality_index        15
split                 train
surname               Totah
Name: 0, dtype: object
char_vocab= <V

### Training loop

In [14]:
mask_index = vectorizer.char_vocab.mask_index

model = model.to(args.device)


optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)
train_state = make_train_state(args)

epoch_bar = tqdm_notebook(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm_notebook(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm_notebook(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)

try:
    for epoch_index in range(1):#args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        model.train()
        
        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------    
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = model(x_in=batch_dict['x_data'])

            # step 3. compute the loss
            loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)


            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the  running loss and running accuracy
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss,
                                  acc=running_acc,
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        model.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # compute the output
            y_pred = model(x_in=batch_dict['x_data'])

            # step 3. compute the loss
            loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)

            # compute the  running loss and running accuracy
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            # Update bar
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=model, 
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break
        
        # move model to cpu for sampling
        model = model.cpu()
        print("#################################")
        sampled_surnames = decode_samples(
            sample_from_model(model, vectorizer, num_samples=2), 
            vectorizer)
        epoch_bar.set_postfix(sample1=sampled_surnames[0], 
                              sample2=sampled_surnames[1])
        # move model back to whichever device it should be on
        model = model.to(args.device)
        
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
except KeyboardInterrupt:
    print("Exiting loop")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


串流輸出內容已截斷至最後 5000 行。
y_out_5.shape torch.Size([2, 19, 88])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
__getitem__
surname= Nawabe
__getitem__
surname= Aoki
x_in.shape= torch.Size([2, 19])
x_embedded.shape= torch.Size([2, 19, 32])
y_out_1.shape torch.Size([2, 19, 32])
_.shape torch.Size([1, 2, 32])
y_out_2.shape torch.Size([38, 32])
y_out_3.shape torch.Size([38, 88])
y_out_5.shape torch.Size([2, 19, 88])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
__getitem__
surname= Tabuchi
__getitem__
surname

In [15]:
np.random.choice(np.arange(len(vectorizer.nationality_vocab)), replace=True, size=2)

array([8, 7])

In [16]:
# compute the loss & accuracy on the test set using the best available model

model.load_state_dict(torch.load(train_state['model_filename']))

model = model.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_acc = 0.
model.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = model(x_in=batch_dict['x_data'])

    # compute the loss
    loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)

    # compute the accuracy
    running_loss += (loss.item() - running_loss) / (batch_index + 1)

    acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss 
train_state['test_acc'] = running_acc 

串流輸出內容已截斷至最後 5000 行。
y_out_5.shape torch.Size([2, 19, 88])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
__getitem__
surname= Paternoster
__getitem__
surname= Ganza
x_in.shape= torch.Size([2, 19])
x_embedded.shape= torch.Size([2, 19, 32])
y_out_1.shape torch.Size([2, 19, 32])
_.shape torch.Size([1, 2, 32])
y_out_2.shape torch.Size([38, 32])
y_out_3.shape torch.Size([38, 88])
y_out_5.shape torch.Size([2, 19, 88])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
y_pred.size()= torch.Size([2, 19, 88]) y_true.size()= torch.Size([2, 19])
y_pred.size() out = torch.Size([38, 88]) y_true.size()= torch.Size([38])
__getitem__
surname= Acerbi
__getitem__
su

In [17]:
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

Test loss: 2.7830586194992075;
Test Accuracy: 22.220017341340927


## Inference

In [18]:
# number of names to generate
num_names = 10
model = model.cpu()
# Generate nationality hidden state
sampled_surnames = decode_samples(
    sample_from_model(model, vectorizer, num_samples=num_names), 
    vectorizer)
# Show results
print ("-"*15)
for i in range(num_names):
    print (sampled_surnames[i])

串流輸出內容已截斷至最後 5000 行。
        [23],
        [23],
        [23],
        [ 3],
        [14],
        [10],
        [ 7]])
 x_emb_t= torch.Size([10, 1, 32])
 rnn_out_t= torch.Size([10, 1, 32])
h_t= torch.Size([1, 10, 32])
 prediction_vector= torch.Size([10, 88])
 probability_vector= torch.Size([10, 88])
probability_vector= tensor([[2.0515e-05, 1.4184e-05, 1.8504e-05, 9.5668e-02, 1.8837e-04, 1.4479e-01,
         1.5088e-02, 1.8696e-01, 9.7646e-03, 2.0740e-04, 7.4169e-03, 7.2512e-02,
         8.6127e-03, 1.4881e-04, 8.6201e-03, 3.2918e-02, 3.0941e-02, 3.5017e-04,
         1.4923e-01, 6.8624e-03, 1.4559e-04, 9.0500e-03, 2.3700e-04, 1.1060e-01,
         4.3081e-04, 1.2723e-02, 2.4749e-04, 3.7166e-02, 3.3610e-03, 2.2348e-04,
         1.9817e-04, 1.9892e-02, 2.2507e-04, 1.0667e-02, 2.6429e-04, 1.5586e-04,
         2.1258e-03, 1.6876e-04, 1.2586e-04, 2.4248e-03, 1.2803e-04, 1.4293e-04,
         1.2001e-04, 1.1386e-02, 1.4433e-04, 5.5081e-05, 9.6738e-05, 9.2443e-05,
         5.6838e-05, 5.7549e-0